In [2]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

import os 
import pandas as pd
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt

import sys
sys.path.append("G:/My Drive/WORKING_MEMORY/EXPERIMENTS/ELECTROPHYSIOLOGY/ANALYSIS/src/functions/")
import ephys_functions as ephys
import model_functions as mod
import behavioral_functions as beh

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings. filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def repeat_reward_side(row):
    '''

    '''
    # Compare the current response with the previous one. If that matches, return a 1 meaning it repeated. 
    if row['trials'] != 0:
        # Compare the current response with the previous one. .  
        if row['reward_side'] == row['previous_reward_side']:
            # if it matched and the answer was 1, it means that it repeated a right response
            if row['reward_side'] == 1:
                return 2
            # if it matched and the answer was 0, it means that it repeated a left response
            else:
                return 1
        # Alternations
        else:
            return 0
    else:
        return np.nan

In [ ]:
path = 'G:/My drive/WORKING_MEMORY/PAPER/WM_manuscript_FIGURES/Data/'
file_name = 'global_behavior_10_paper'
df = pd.read_csv(path+file_name+'.csv', index_col=0)

In [ ]:
from scipy.stats import linregress

fig, axes = plt.subplots(1,2,  figsize=(10,5))
groupings=['subject','session']
df_results = pd.DataFrame()
df_results['repeat_choice'] = 0.5* df.loc[(df['repeat_choice_side']==1)].groupby(groupings)['valids'].count()/df.loc[(df.vector_answer == 0)].groupby(groupings)['valids'].count() + 0.5*df.loc[(df['repeat_choice_side']==2)].groupby(groupings)['valids'].count()/df.loc[(df.vector_answer == 1)].groupby(groupings)['valids'].count()
df_results['accuracy'] = df.groupby(groupings)['hit'].mean()
df_results['accuracy_0'] = df.loc[df.delay_times == 0.01].groupby(groupings)['hit'].mean()
df_results.reset_index(inplace=True)
palette = sns.color_palette(['black'], len(df.subject.unique()))

ax=axes[0]
sns.scatterplot(x='accuracy',y='repeat_choice',hue='subject',data=df_results, legend=False, palette = 'Greys', ax=ax)
slope, intercept, r_value, p_value, std_err = linregress(df_results['repeat_choice'], df_results['accuracy'])
# Plot the regression line
ax.plot(df_results['accuracy'], intercept + slope * df_results['accuracy'], color='red', label=f'R={r_value:.2f}, p={p_value:.2e}')
ax.legend()

ax.set_ylabel('Accuracy')
ax.set_xlabel('Repeat choice')

ax=axes[1]
sns.scatterplot(x='accuracy',y='accuracy_0',hue='subject',data=df_results, legend=False, palette = 'Greys', ax=ax)
slope, intercept, r_value, p_value, std_err = linregress(df_results['accuracy'], df_results['accuracy_0'])
ax.plot(df_results['accuracy'], intercept + slope * df_results['accuracy'], color='red', label=f'R={r_value:.2f}, p={p_value:.2e}')
ax.set_xlabel('Lapse rate')
ax.set_ylabel('Accuracy')
plt.legend()
sns.despine()
plt.show()
